Last update 03.08.2020

Author : Jeanne Colbois

Please send any comments, questions or remarks to Jeanne Colbois: jeanne.colbois@epfl.ch.
The author would appreciate to be cited in uses of this code, and would be very happy to hear about developments.

In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
# Import modules
import numpy as np # maths

from time import time# timer for debug
    # gather results obtained from the runs  

import KagomeFunctions_OBC as kf # "library"< allowing to work on Kagome
import KagomeDrawing as kdraw # "library" allowing to plot Kagome
import KagomeFT as kft
import AnalysisFunctions_OBC_XPEEM as af # functions to make the analysis code cleaner
    #plots using matplotlib
import matplotlib
import matplotlib.pyplot as plt
import StartStates as strst
#import matplotlib.image as mpimg
%matplotlib inline
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams.update({'font.size': 14})

In [3]:
secondfoldername = ['../../Analysis_PSI/ExperimentalDataAndAnalysis_August2020/Inputs/']*11
secondfilename = ['PEEM_spin_config_threshold_0.txt',
                  'PEEM_spin_config_threshold_0.1.txt',
                  'PEEM_spin_config_threshold_0.2.txt',
                  'PEEM_spin_config_threshold_0.3.txt',
                  'PEEM_spin_config_threshold_0.4.txt',
                  'PEEM_spin_config_threshold_0.5.txt',
                  'PEEM_spin_config_threshold_0.6.txt',
                  'PEEM_spin_config_threshold_0.7.txt',
                  'PEEM_spin_config_threshold_0.8.txt',
                  'PEEM_spin_config_threshold_0.9.txt',
                  'PEEM_spin_config_threshold_1.txt']

In [6]:
mlist = np.array(mlist)
print(mlist)
avgm = np.sum(mlist)/len(mlist)
print(np.std(mlist))

[-0.03806584 -0.03775241 -0.03419689 -0.03352412 -0.02431907 -0.02311796
 -0.02318624 -0.01685393 -0.01654846  0.00940439 -0.01265823]
0.01320636902092263


In [7]:
avgm

-0.022801706379208773

#### Bootstrapping the result to get an error estimate:
The idea is to compute the correlations as the average of the correlations over 18 of the 20 samples, chosing these 18 in an arbitrary way, then compute the error as the standard deviation of this result

In [8]:
n = len(mlist)

In [9]:
np.array(covlist).shape

(11, 4)

In [10]:
d = np.array(dlist).sum(0)/n
vard = np.array(vardlist).sum(0)/n

cov = np.array(covlist).sum(0)/n
#varcov = np.array([(covlist[i] - cov)**2 for i in range(n)]).sum(0)/n
#
#varcov2 = np.array(varcovlist).sum(0)/n

In [11]:
covlist

[[0.3907932042842885,
  0.10680633931552005,
  0.12732330866464475,
  0.08288977030627966],
 [0.4897820007312742,
  0.13260117617656822,
  0.14443223845442899,
  0.0879650810649079],
 [0.6026942245866573,
  0.1636307265053883,
  0.16902327317186272,
  0.11430707657122371],
 [0.7206051854108811,
  0.2072894698678067,
  0.19744699106401573,
  0.13794556680100634],
 [0.7904871682477738,
  0.2378405176912651,
  0.20976282441480723,
  0.1830160509041898],
 [0.8742249154934969,
  0.2514251953623093,
  0.21443865412937757,
  0.21714439821236037],
 [0.9392142277060839,
  0.3028885354149664,
  0.24243585182752503,
  0.2526766595289078],
 [0.9754109495992335,
  0.3584914032168609,
  0.26325953151891623,
  0.30805975334375624],
 [0.9919236080325707,
  0.38186021880368404,
  0.3109821111242755,
  0.34976452119309187],
 [0.9968204863796495,
  0.43136510347446383,
  0.39730493371006387,
  0.37945256842894703],
 [0.9958082497212903,
  0.5362078496406859,
  0.488014311270125,
  0.38755980861244044]]

In [ ]:
avgcorr = []
corr = np.array(covlist)
s = 5
nsample = 20
masks = []
for randid in range(nsample):
    mask = np.random.choice(20, size=s)
    masks.append(mask)
    avgcorr.append(corr[mask,:,:].sum(0)/s)
    
avgcorr = np.array(avgcorr)
navg = avgcorr.shape[0]
stdev = np.std(avgcorr, axis=0)
avgcorrplot = avgcorr.sum(0)/nsample

In [ ]:
stdev

In [ ]:
avgcorrplot

In [ ]:
ListNei = ['1', '2', '3par', '3star', '4', '5', '6', '6bis', '7', '8', '8bis','9']
fig, ax = plt.subplots(figsize = (8,8), dpi=200)
for index in range(4):
    print(index)
    plt.errorbar(1, avgcorrplot[index,0],yerr=stdev[index,0], 
    label='Neighbour {0}'.format(ListNei[index]),  fmt= 'x')
plt.xlabel('')
plt.ylabel('Accumulated correlations')
plt.title('20 samples average,\n errors from std deviation of {0} samples averages'.format(s))
plt.ylim(-0.27, 0.12)
plt.legend()
plt.savefig(secondfoldername[0]+'AvgFirstFewCorrelations'+secondfilename[0]+'.png')    

In [ ]:
fig, ax = plt.subplots(figsize = (8,8), dpi=200)
for i in range(5):
    plt.gca().set_prop_cycle(None)
    for index in range(4):
        print(index)
        if i == 0:
            plt.errorbar(i, avgcorr[i,index,0],yerr=stdev[index,0], 
            label='Neighbour {0}'.format(ListNei[index]),  fmt= 'x')
        else:
            plt.errorbar(i, avgcorr[i,index,0],yerr=stdev[index,0],  fmt= 'x')
plt.xticks([0, 1, 2, 3, 4], ['Set 1', 'Set 2', 'Set 3','Set 4','Set 5'])
plt.ylabel('Accumulated correlations')
plt.ylim(-0.27, 0.12)
plt.legend()
plt.savefig(secondfoldername[0]+'AvgFirstFewCorrelations'+'_5Masks_{1}_samples_bootstrapping'.format(i,s)+secondfilename[0]+'.png')    

In [ ]:
print(cov)

In [ ]:
print(np.sqrt(np.array(varcov)))
print(np.sqrt(np.array(varcov2)))

In [ ]:
fig, ax = plt.subplots(dpi=200)
plt.errorbar(d, np.array(cov),xerr = np.sqrt(np.array(vard)),yerr=np.sqrt(np.array(varcov)),  fmt= '.')
plt.xlabel('Distance')
plt.ylabel('Accumulated correlations')
plt.ylim(-0.27,0.12)
plt.savefig(secondfoldername[i] + 'FirstResult_' + secondfilename[i]+'.png')
plt.show()

In [ ]:
ListNei = ['1', '2', '3par', '3star', '4', '5', '6', '6bis', '7', '8', '8bis','9']
for i in range(n):
    fig, ax = plt.subplots(figsize = (8,8), dpi=200)

    for index, covi in enumerate(covlist[i][0:5]):
        #print(index)
        plt.errorbar(1, covi,yerr=np.sqrt(np.array(varcovlist[i])[index]), label='Neighbour {0}'.format(ListNei[index]),  fmt= 'x')

    plt.xlabel('')
    plt.xticks([])
    plt.ylabel('Accumulated correlations')
    plt.ylim(-0.27, 0.12)
    plt.legend(loc='best')
    plt.savefig(secondfoldername[i]+'FirstFewCorrelations'+secondfilename[i]+'.png')

fig, ax = plt.subplots(figsize = (8,8),dpi=200)
x = [1 for y in cov];

for index, covi in enumerate(cov[0:4]):
    print(index)
    plt.errorbar(1, covi,yerr=np.sqrt(np.array(varcov2)[index]), label='Neighbour {0}'.format(ListNei[index]),  fmt= 'x')
plt.xlabel('')
plt.xticks([])
plt.ylabel('Accumulated correlations (20 samples average)')
plt.ylim(-0.27, 0.12)
plt.legend()
plt.savefig(secondfoldername[0]+'AvgFirstFewCorrelations'+secondfilename[0]+'.png')    

In [ ]:
#ssecondfoldername = '../Analysis_PSI/Kagome_54-1_Demag_all/'
#ssecondfilename = '54-1_Demag7_5-TR_spin_config.txt'
#iimgfile = ""
#ffactor = 178.7
#cconfiglist, x, y, sconf = af.LoadSpinConfigsLarge(secondfoldername, secondfilename, imgfile, alpha = 1, factor = factor)
#pplt.plot(x[43]/factor, y[43]/factor,'.', color = "red")
#pplt.plot(x[42]/factor, y[42]/factor,'.', color = "blue")
#pprint(x[43]," ", y[10], " ", x[11], " ", y[11])
#pprint(np.sqrt((x[43]-x[42])**2 + (y[43]-y[42])**2))
#mm = sum(sconf)/len(sconf)
#pprint(m)